This notebook preprocessed raw trisonica data to adjust the reference system to the EP one 

In [ ]:
from pathlib import Path
import numpy as np
from multiprocessing import Pool
import logging as log
from functools import partial
from wind_tools import *
from scipy.spatial.transform import Rotation as R
from copy import deepcopy
log.basicConfig(level=log.INFO) #uncomment to see messages from extract
# name_re = r"*_TRS_M00506_com3.raw"
u, v, w = 0, 1, 2 #defing costants for accessing data

In [ ]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (18,10)

In [ ]:
# in_dir = Path("2020_data/data_20200724_final_field_install/raw")
# out_dir = Path("2020_data/data_20200724_final_field_install/preprocessed")

In [ ]:
in_dir = Path("2020_data/data_field_v2_from_20208010/raw")
out_dir = Path("2020_data/data_field_v2_from_20208010/preprocessed")

## Filtered wind directions

In [ ]:
start_angle = 250
range_angle = 30
wm_basename = '_WM_174605_com1.csv'

def wm_dir_filter(f):
    """f is the filename of another anemometer at the same time"""
    wm_path = out_dir / (f.name[:13] + wm_basename) # the first 13 is the date info
    # Reads already processed (and rotated) WM
    wm = pd.read_csv(wm_path)
    wd = get_wind_dir(wm.u, wm.v)
    return filter_by_wind_dir_single(wd, start_angle, range_angle) 

def get_filtered_proc(filt):
    def _inner(data):
        new_data = data
        length = min(len(data), len(filt))
        new_data[:length][~filt[:length ]] = -9999
        return new_data
    return _inner

def extract_filtered(setting, f):
    
    filt = wm_dir_filter(f)
    old_processor = deepcopy(getordef(setting, 'transform'))
    def my_transform(setg, data):
        data = old_processor(setg, data)
        return get_filtered_proc(filt)(data)
    setting['transform'] = my_transform
    extract(setting,f)


### General funcs

In [ ]:
def extract(setting, f):
    usecols, delimiter, name_suffix, transform, replace = map(partial(getordef, setting), ['usecols', 'delimiter', 'name_suffix', 'transform', 'replace'])
    # log.info(f"opening {f}")
    out_name = out_dir / f"{f.name[:-4]}{name_suffix}.csv"
    if not replace and out_name.exists():
        log.debug(f" exists, skipping {out_name}")
    else:
        data = np.genfromtxt(f, usecols=usecols, invalid_raise=False, delimiter=delimiter)
        data = transform(setting, data)
        np.savetxt(out_name, data, header="u,v,w,t", delimiter=',', fmt='%2.2f', comments='')
        log.info(f"saved file {out_name}")

def nothing(setting, x):
    return x

def getordef(setg, item):
    if item in setg:
        return setg[item]
    elif item in settings_default:
        return settings_default[item]
    else:
        raise KeyError("Key not found in default settings")

In [ ]:
def rotate(setting, data):
    rot = getordef(setting, 'rotation')
    rot_data = data.copy() #to preserve temperature
    rot_data[:, [0,1,2]] = rot.apply(data[:, [0,1,2]])
    
    return rot_data

In [ ]:
def main(parallel=True):
    print("starting processing...")
    if not out_dir.is_dir(): out_dir.mkdir(parents=True, exist_ok=True)
    with Pool() as p:
        for key in settings:
            print(f"processing {key}")
            proc = partial(getordef(settings[key], 'extractor'), settings[key])
            if parallel:
                p.map(proc, in_dir.glob(getordef(settings[key], "name_re")))
            else:
                list(map(proc, in_dir.glob(getordef(settings[key], "name_re"))))
    print("done")
    # for f in in_dir.glob(name_re):
    #     extract(f)

## Settings

settings for each anemometer, the processor take in input an array where columns are u,v,w,t and returns a transformed array

In [ ]:
settings_default = {
        
        'extractor': extract, # Function that extracts from the text file the data array and saves it to the disk
        'usecols': None, # columns that should be considered by extract function
        'delimiter': None, #delimi tween column in the input file
    
        'transform': nothing, # function to tranform the data before saving like rotating or filtering. signature is process(setting, data) -> transformed data
        'rotation': None, # scipy rotation for the rotate function      
    
        'name_suffix': '', #suffix that is appened on the file name before saving
        'replace': False, # if should replace exiting file with same name        
    }

In [ ]:
rot_m507 = R.from_euler('XYZ', [90, -250,  135], degrees=True) # 250 can be +/- 10 °

rot_m506 = R.from_euler('z', [-90.], degrees=True)
rot_wm1 = R.from_euler('z', [50], degrees=True) # why here 50 and there -90 bohhh but who cares basta che va


wm_cols = (2,3, 4, 6)
trs_cols = (10, 12, 14, 16)

settings = {
    'm506': {
        'usecols': trs_cols,
        'name_re': r"*_TRS_M00506_com3.raw",
        'transform': rotate,
        'rotation': rot_m506,
        },
    'm507': {
        'usecols': trs_cols,
        'name_re': r"*_TRS_M00507_com2.raw",
        'transform': rotate,
        'rotation': rot_m507
    },
    'm506_raw': {
        'usecols': trs_cols,
        'name_re': r"*_TRS_M00506_com3.raw",
        'name_suffix': "_raw",
        'transform': nothing,
        },
    'wm1':{
        'usecols': wm_cols,
        'name_re': r"*_WM_174605_com1.raw",
        'delimiter': ',',
        'transform': rotate,
        'rotation': rot_wm1,        
    },

#     'm507_raw': { 
#         'usecols': (10, 12, 14, 16),
#         'name_re': r"*_TRS_M00507_com2.raw",
#         'processor': nothing,
#         'name_suffix': '_raw'
#     },
    'wm1_filtered':{
        'usecols': wm_cols,
        'name_re': r"*_WM_174605_com1.raw",
        'extractor': extract_filtered,
        'transform': rotate,
        'rotation': rot_wm1,
        'name_suffix': '_filtered_250_30',
        'delimiter': ',',
        'replace': True,
    },
    'm507_filtered':{
        'usecols': trs_cols,
        'name_re': r"*_TRS_M00507_com2.raw",
        'extractor': extract_filtered,
        'transform': rotate,
        'rotation': rot_m507,
        'name_suffix': '_filtered_250_30',
    },

}

In [ ]:
main(parallel=True)

starting processing...


INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:numexpr.utils:NumExpr defaulting to 8 t

processing m506
processing m507
processing m506_raw
processing wm1
processing wm1_filtered


INFO:root:saved file 2020_data/data_field_v2_from_20208010/preprocessed/20200812-0500_WM_174605_com1_filtered_250_30.csv
INFO:root:saved file 2020_data/data_field_v2_from_20208010/preprocessed/20200812-1330_WM_174605_com1_filtered_250_30.csv
INFO:root:saved file 2020_data/data_field_v2_from_20208010/preprocessed/20200810-1900_WM_174605_com1_filtered_250_30.csv
INFO:root:saved file 2020_data/data_field_v2_from_20208010/preprocessed/20200812-2000_WM_174605_com1_filtered_250_30.csv
INFO:root:saved file 2020_data/data_field_v2_from_20208010/preprocessed/20200813-0430_WM_174605_com1_filtered_250_30.csv
INFO:root:saved file 2020_data/data_field_v2_from_20208010/preprocessed/20200812-1500_WM_174605_com1_filtered_250_30.csv
INFO:root:saved file 2020_data/data_field_v2_from_20208010/preprocessed/20200813-0830_WM_174605_com1_filtered_250_30.csv
INFO:root:saved file 2020_data/data_field_v2_from_20208010/preprocessed/20200812-1400_WM_174605_com1_filtered_250_30.csv
INFO:root:saved file 2020_data/d

processing m507_filtered
done


In [ ]:
rot_m506.apply(np.array([[1,2,3]]))

# Time to refactor

## Utiliy funcs

In [ ]:
def load(path, usecols, delimiter=None):
    return np.genfromtxt(path, usecols=usecols, delimiter=delimiter, invalid_raise=False)

def save(data, file)

## My Trisonica (of FrankeStonica)

# Tests and Explanations

In [ ]:
def vecs3d_legend(name="", colors=['r', 'g', 'b', 'fuchsia', 'yellow', 'cyan'], labels=['u', 'v', 'w', 'u reference system', 'v reference system', 'w reference system']):
    handles = []
    for col, label in zip(colors, labels):
        handles.append(mpatches.Patch(color=col, label=label))
    
    plt.legend(handles=handles)

###  M506

In [ ]:
rot_m506 = R.from_euler('z', [-90.], degrees=True)
def process_m506(data):
    # fix TRS1 u and v ax by inverting it, due to different coordinate system between trisonica and EP
    rot_data = data.copy() #to preserve temperature
    rot_data[:, [u,v,w]] = rot_m506.apply(data[:, [u,v,w]])
    return data

In [ ]:
ax = plt.figure().add_subplot(111, projection='3d')
ax.set_title("M507 reference frame compared to the world one ")
plot_vecs3d(rot_m506.apply(v0), colors=['r', 'g', 'b'], ax=ax, lw=5 )
plot_vecs3d(v0, colors=['fuchsia', 'yellow', 'cyan'], ax=ax, lw=2) # plot origins as reference only on the last one

vecs3d_legend()

#### Tests

In [ ]:
v1 = [1,2,3]

In [ ]:
test_close(rot_m506.apply(v1),[ 2., -1.,  3.] )

### M507

In [ ]:
rot_m507 = R.from_euler('XYZ', [90, 0,  135], degrees=True) 

def process_m507(data):      
    rot_data = data.copy() #to preserve temperature
    rot_data[:, [u,v,w]] = rot_m507.apply(data[:, [u,v,w]])
    
    return rot_data


In [ ]:
ax = plt.figure().add_subplot(111, projection='3d')
ax.set_title("M507 reference frame compared to the world one ")
plot_vecs3d(rot_m507.apply(v0), colors=['r', 'g', 'b'], ax=ax, lw=5 )
plot_vecs3d(v0, colors=['fuchsia', 'yellow', 'cyan'], ax=ax, lw=2) # plot origins as reference only on the last one

vecs3d_legend()

**Those tests works only no rotation on the y** TODO fix this

In [ ]:
v2 = [1,1,2],

In [ ]:
rot_m507.apply(v2)

In [ ]:
test_close(rot_m507.apply(v2), [-np.sqrt(2), -2., 0.])

In [ ]:
v3 = [0,1,3]

In [ ]:
test_close(rot_m507.apply(v3), [-np.sin(np.pi/4),-3,-np.sin(np.pi/4)])

## Testing M507 wind filter

In [ ]:
#interesting period with EP data is
testf = in_dir / "20200724-2200_WM_174605_com1.raw"
tfilt = wm_dir_filter(testf)

In [ ]:
tfilt[:1210]

In [ ]:
ep_outf = in_dir / "../processed/eddypro_raw_datasets/level_1/20200724-2202_raw_dataset_2020-08-07T163138_adv.txt"

In [ ]:
ep_out = np.genfromtxt(ep_outf, skip_header=10)

In [ ]:
ep_out[ep_out==-9999.] = np.nan

In [ ]:
ep_out[181]

In [ ]:
wd = mod(get_wind_dir(ep_out[:, 0], ep_out[:, 1]) + wm_offset)

In [ ]:
wdr = get_wind_dir(ep_out[:, 0], ep_out[:, 1])

In [ ]:
wdr[~np.isnan(wdr)].mean()

In [ ]:
wd[~np.isnan(wd)].mean()